<a href="https://colab.research.google.com/github/maximematerno/DS-Unit-1-Sprint-1-Dealing-With-Data/blob/master/LS_DS_111_A_First_Look_at_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science - A First Look at Data



## Lecture - let's explore Python DS libraries and examples!

The Python Data Science ecosystem is huge. You've seen some of the big pieces - pandas, scikit-learn, matplotlib. What parts do you want to see more of?

In [0]:
2+2


4

## Assignment - now it's your turn

Pick at least one Python DS library, and using documentation/examples reproduce in this notebook something cool. It's OK if you don't fully understand it or get it 100% working, but do put in effort and look things up.

##Reinhart-Rogoff replication¶


In [0]:
import statsmodels.api as sm
import patsy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

RR = pd.read_csv('https://gist.githubusercontent.com/vincentarelbundock/5409893/raw/a623f2f3bae027a0e51dd01ac5b70d44d909a7b9/RR-processed.csv')

### I had a little difficulty to read the csv file

In [21]:
RR.groupby('Country').size()


Country
Australia      64
Austria        62
Belgium        63
Canada         64
Denmark        56
Finland        64
France         54
Germany        59
Greece         40
Ireland        63
Italy          59
Japan          54
Netherlands    53
New Zealand    64
Norway         64
Portugal       58
Spain          42
Sweden         64
UK             64
US             64
dtype: int64

## Bins

In [0]:
bins = ["0-30%","30-60%","60-90%","Above 90%"]
RR['dgcat'] = np.digitize(RR.debtgdp, [0,30,60,90,np.inf]) - 1
RR.dgcat = [bins[x] for x in RR.dgcat]

bins = ["0-30%","30-60%","60-90%","90-120%","Above 120%"]
RR['dgcat2'] = np.digitize(RR.debtgdp, [0,30,60,90,120,np.inf]) - 1
RR.dgcat2 = [bins[x] for x in RR.dgcat2]

## Regression analysis

In [28]:
y,X = patsy.dmatrices('dRGDP ~ dgcat', data=RR[['dRGDP', 'dgcat']].dropna())

print(sm.OLS(y,X).fit().summary())

                            OLS Regression Results                            
Dep. Variable:                  dRGDP   R-squared:                       0.045
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     18.36
Date:                Mon, 05 Aug 2019   Prob (F-statistic):           1.22e-11
Time:                        21:12:31   Log-Likelihood:                -2927.9
No. Observations:                1175   AIC:                             5864.
Df Residuals:                    1171   BIC:                             5884.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              4.1735      0

In [29]:
y2,X2 = patsy.dmatrices('dRGDP ~ dgcat2', data=RR[['dRGDP', 'dgcat2']].dropna())
print(sm.OLS(y2,X2).fit().summary())

                            OLS Regression Results                            
Dep. Variable:                  dRGDP   R-squared:                       0.046
Model:                            OLS   Adj. R-squared:                  0.043
Method:                 Least Squares   F-statistic:                     14.24
Date:                Mon, 05 Aug 2019   Prob (F-statistic):           2.36e-11
Time:                        21:13:10   Log-Likelihood:                -2926.9
No. Observations:                1175   AIC:                             5864.
Df Residuals:                    1170   BIC:                             5889.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                4.1735 

## Table 3 Corrected

In [30]:
RR.dRGDP.groupby(RR.dgcat).mean()

dgcat
0-30%        4.173523
30-60%       3.092145
60-90%       3.186575
Above 90%    2.167972
Name: dRGDP, dtype: float64

In [31]:
RR.dRGDP.groupby([RR.Country, RR.dgcat]).mean().unstack()

dgcat,0-30%,30-60%,60-90%,Above 90%
Country,,,,
Australia,3.205885,4.947205,4.042175,3.774250
Austria,5.207527,3.256526,-3.824000,NaN
Belgium,NaN,4.191655,3.079868,2.566828
Canada,2.515704,3.525446,4.523574,2.956640
Denmark,3.518584,1.700034,2.391147,NaN
Finland,3.817029,2.418535,5.488887,NaN
France,5.058476,2.616159,3.019631,NaN
Germany,3.873759,0.875803,NaN,NaN
Greece,4.001282,0.340200,2.696000,2.910632


In [32]:
RR.dRGDP.groupby(RR.dgcat2).mean()

dgcat2
0-30%         4.173523
30-60%        3.092145
60-90%        3.186575
90-120%       2.405934
Above 120%    1.561553
Name: dRGDP, dtype: float64

In [33]:
RR.dRGDP.groupby([RR.Country, RR.dgcat2]).mean().unstack()

dgcat2,0-30%,30-60%,60-90%,90-120%,Above 120%
Country,,,,,
Australia,3.205885,4.947205,4.042175,6.920201,2.987763
Austria,5.207527,3.256526,-3.824000,NaN,NaN
Belgium,NaN,4.191655,3.079868,2.702629,-0.692378
Canada,2.515704,3.525446,4.523574,4.544839,0.574341
Denmark,3.518584,1.700034,2.391147,NaN,NaN
Finland,3.817029,2.418535,5.488887,NaN,NaN
France,5.058476,2.616159,3.019631,NaN,NaN
Germany,3.873759,0.875803,NaN,NaN,NaN
Greece,4.001282,0.340200,2.696000,2.910632,NaN


## Selective treatment of early years

In [34]:
idx = (RR.Country == 'New Zealand') & (RR.Year < 1950) | (RR.Country == 'Australia') & (RR.Year < 1951) | (RR.Country == 'Canada') & (RR.Year < 1951) 
RR_selective = RR[idx == False]
RR_selective.dRGDP.groupby(RR_selective.dgcat).mean()

dgcat
0-30%        4.173523
30-60%       3.092145
60-90%       3.186575
Above 90%    1.919934
Name: dRGDP, dtype: float64

## Equal weights

## Table 3 Weights,Exclusion

In [35]:
RR_selective.mean()

Unnamed: 0    5.916408e+02
Year          1.979633e+03
Debt          1.621458e+07
RGDP          2.369173e+05
GDP           1.957611e+05
dRGDP         3.408270e+00
GDPI          5.034180e+01
GDP1          1.470725e+07
GDP2          1.824865e+07
RGDP1         1.425590e+07
RGDP2         3.072470e+07
GDPI1         5.589542e+02
GDPI2         8.690705e+01
Infl          5.632643e+00
Debt1         5.625405e+05
Debt2         1.050730e+05
Debtalt       1.006665e+07
GDP2alt       4.450790e+05
GDPalt        2.079659e+06
RGDP2alt      1.033331e+05
debtgdp       4.530375e+01
GDP3          7.689113e+04
GNI           5.156239e+08
lRGDP         2.351587e+05
lRGDP1        1.405853e+07
lRGDP2        3.047113e+07
dtype: float64

### Assignment questions

After you've worked on some code, answer the following questions in this text block:

1.  Describe in a paragraph of text what you did and why, as if you were writing an email to somebody interested but nontechnical.

2.  What was the most challenging part of what you did?

3.  What was the most interesting thing you learned?

4.  What area would you like to explore with more time?




1. I imported  the librairy and the csv file  and  I wrote the code .

2. the most challenging was to find a ways to use the csv file and and parenthese for the print statement 

3. I learned how to analyse the data deeper with Regression Analysis and more

4. I would like to explore some Financial analysis





## Stretch goals and resources

Following are *optional* things for you to take a look at. Focus on the above assignment first, and make sure to commit and push your changes to GitHub (and since this is the first assignment of the sprint, open a PR as well).

- [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/)
- [scikit-learn documentation](http://scikit-learn.org/stable/documentation.html)
- [matplotlib documentation](https://matplotlib.org/contents.html)
- [Awesome Data Science](https://github.com/bulutyazilim/awesome-datascience) - a list of many types of DS resources

Stretch goals:

- Find and read blogs, walkthroughs, and other examples of people working through cool things with data science - and share with your classmates!
- Write a blog post (Medium is a popular place to publish) introducing yourself as somebody learning data science, and talking about what you've learned already and what you're excited to learn more about.